In [1]:
import numpy as np
from simplifiedLikelihoods import Data,UpperLimitComputer
import pandas as pd

### Define MET bins and their order

In [2]:
metBins = [250,  280,  310,  340,  370,  400,  430,  470,  510, 550,  590,  640,  690,  
            740,  790,  840,  900,  960, 1020, 1090, 1160, 1250, 1400]

### Load CMS data

In [3]:
cmsYieldFile = '../AuxInfo/CMS-EXO-20-004-data/HEPData-ins1894408-v2-csv/Simplifiedlikelihood:Yields(Monojet).csv'
bgYields = np.genfromtxt(cmsYieldFile,delimiter=',',skip_header=5,
                          skip_footer=67,names=True,dtype=None,encoding=None)

dataYields = np.genfromtxt(cmsYieldFile,delimiter=',',skip_header=73,
                           names=True,dtype=None,encoding=None)

covarianceFile = '../AuxInfo/CMS-EXO-20-004-data/HEPData-ins1894408-v2-csv/Simplifiedlikelihood:covariancematrix(Monojet).csv'
cov = np.genfromtxt(covarianceFile,delimiter=',',skip_header=5,
                           names=True,dtype=None,encoding='UTF-8')

### Extract covariance matrix

In [4]:
srOrder = cov['Second_Bin'][:66]
covMatrix = np.zeros((len(srOrder),len(srOrder)))
for pt in cov:
    i = np.where(srOrder == pt['First_Bin'])
    j = np.where(srOrder == pt['Second_Bin'])
    covMatrix[i,j] = pt['Covariance']

### Get number of observed and expected (BG) events

In [5]:
nobs = [int(dataYields[dataYields['Bin'] == sr]['Data_yield'][0]) for sr in srOrder]
nbg = [float(bgYields[bgYields['Bin'] == sr]['Background_yield'][0]) for sr in srOrder]

### Load Recast Data

In [6]:
recastFile = '../scanResults.pcl'
recastData = pd.read_pickle(recastFile)

### Get all model points

In [7]:
models = []
mCols = ['Coupling','Mode','$m_{med}$','$m_{DM}$']
for row in recastData[mCols].values:
    m = dict(zip(mCols,row.tolist()))
    if m not in models:
        models.append(m)
print(models)

[{'Coupling': 'Axial', 'Mode': 'DM+QCDjets', '$m_{med}$': 1500.0, '$m_{DM}$': 1.0}, {'Coupling': 'Axial', 'Mode': 'DM+QCDjets', '$m_{med}$': 1750.0, '$m_{DM}$': 1.0}, {'Coupling': 'Axial', 'Mode': 'DM+QCDjets', '$m_{med}$': 2000.0, '$m_{DM}$': 1.0}]


### Loop over model points and compute UL on mu

In [8]:
ulComp = UpperLimitComputer()

for m in models:
    dfModel = recastData.loc[(recastData[list(m)] == pd.Series(m)).all(axis=1)]
    ns = []
    for sr in srOrder:
        dataset,ibin = sr.split('_')[1:]
        dataset = eval(dataset)
        ibin = eval(ibin.replace('bin',''))
        binLabel = 'bin_%1.1f_%1.1f' %(metBins[ibin],metBins[ibin+1])
        signalYield = dfModel[dfModel['Data-takingperiod'] == dataset][binLabel].iloc[0]
        ns.append(signalYield)
    ns = np.array(ns)    
    data = Data(observed=nobs, backgrounds=nbg, covariance=covMatrix, 
                nsignal=ns,deltas_rel=0.0)
    dataExp = Data(observed=[int(b) for b in nbg], backgrounds=nbg, covariance=covMatrix, 
                   nsignal=ns,deltas_rel=0.0)    
    
    ul = ulComp.getUpperLimitOnMu(data)
    ulExp = ulComp.getUpperLimitOnMu(dataExp)    
    recastData.loc[dfModel.index,'$\mu^{UL}_{obs}$'] = ul
    recastData.loc[dfModel.index,'$\mu^{UL}_{exp}$'] = ulExp

In [9]:
recastData

,Coupling,Mode,$m_{med}$,$m_{DM}$,Data-takingperiod,Luminosity (1/fb),bin_250.0_280.0,bin_250.0_280.0_ErrorPlus,bin_250.0_280.0_ErrorMinus,bin_280.0_310.0,...,Tauveto,Bjetveto,Photonveto,"$\Delta \phi (jet,p_{T}^{miss})>0.5$ rad",LeadingAK4jet$p_{T}>100$GeV,LeadingAK4jet$\eta<2.4$,HCALmitigation(jets),HCALmitigation($\phi^{miss}$),$\mu^{UL}_{obs}$,$\mu^{UL}_{exp}$
0,Axial,DM+QCDjets,1500.0,1.0,2018,59.7,702.329838,38.409060,38.409060,541.132292,...,0.374582,0.326757,0.318788,0.318257,0.314939,0.313974,0.304742,0.267150,0.503369,0.244855
1,Axial,DM+QCDjets,1500.0,1.0,2017,41.5,566.387221,33.501225,33.501225,503.851812,...,0.382359,0.339013,0.328340,0.328054,0.326842,0.326208,0.326208,0.326208,0.503369,0.244855
2,Axial,DM+QCDjets,1500.0,1.0,2016,36.0,558.428815,34.127560,34.127560,420.663402,...,0.384592,0.341331,0.334650,0.334413,0.331094,0.329966,0.329966,0.329966,0.503369,0.244855
3,Axial,DM+QCDjets,1750.0,1.0,2018,59.7,392.468875,23.069178,23.069178,297.622544,...,0.388127,0.341684,0.333516,0.332704,0.329350,0.328672,0.315988,0.284404,0.880853,0.437982
4,Axial,DM+QCDjets,1750.0,1.0,2017,41.5,289.727317,18.770297,18.770297,262.953360,...,0.399420,0.348820,0.338700,0.338294,0.336990,0.335732,0.335732,0.335732,0.880853,0.437982
5,Axial,DM+QCDjets,1750.0,1.0,2016,36.0,259.569320,17.917094,17.917094,224.165738,...,0.384324,0.340732,0.327463,0.327363,0.325285,0.323937,0.323937,0.323937,0.880853,0.437982
6,Axial,DM+QCDjets,2000.0,1.0,2018,59.7,154.744635,9.415424,9.415424,153.050999,...,0.380058,0.336623,0.327248,0.326841,0.325122,0.324267,0.311418,0.275559,1.574081,0.733794
7,Axial,DM+QCDjets,2000.0,1.0,2017,41.5,146.250221,10.758984,10.758984,120.701936,...,0.366852,0.321657,0.315166,0.314900,0.312753,0.311006,0.311006,0.311006,1.574081,0.733794
8,Axial,DM+QCDjets,2000.0,1.0,2016,36.0,137.196545,10.370482,10.370482,109.592453,...,0.379348,0.333107,0.324076,0.323460,0.321491,0.319851,0.319851,0.319851,1.574081,0.733794


In [10]:
recastData.to_pickle(recastFile)